In [1]:
import os
import sys
import time
import pickle
from openfermion import qubit_operator_to_pauli_sum

from kcommute.sorted_insertion import get_si_sets, get_si_sets_v2, get_terms_ordered_by_abscoeff
from kcommute.sorted_insertion_v2_parallel import get_si_sets_v2_parallel
from Hamiltonians.load_h2o import load_h2o_hamiltonian
from Hamiltonians.load_owp import load_owp_hamiltonian

This notebook assesses the validity and timing benchmarks of a pair of refactors which modify get_si_sets from the original kcommute repository: https://github.com/rmlarose/kcommute/tree/main/kcommute .  

At the end of this notebook it shows how one would use this code to compute the groupings for the OWP Hamiltonian as required for future hardware work: https://github.com/rmlarose/calibrate-ibm .  To perform this optimally, please request 17 cores and sufficient memory, else switch to the single core serial method (roughly 3x more time) which has been commented out.

The serial method get_si_sets_v2 benefits from utilizing the symplectic form representation.  This enables relevant operations to be implemented as simple bitwise operations and parity checks.  Some additional optimizations were made based on the cheapness of 1-commuting checks in this formalism, as well as using a 1-commuting representative for each group.  

The parallel method get_si_sets_v2_parallel utilizes the same data structure and methods but depends on several further optimizations to squeeze out another ~3x speedup (heavily dependent on number of cores and parameters.  Suggested parameters chosen at the end of the notebook).  Parallel workers check for incompatibility with existing groups.  Main process maintains authority over the grouping and sorted insertion ordering is maintained.  Workers operate in a 'leapfrog' fashion checking whether strings in their chunk are incompatible with existing groupings.  Existing groupings and incompatibilities are handshaked from the parent process intermittently.  Tuning the chunk size and frequency is non-trivial and exhibits a non-monotonic tradeoff between information movement and structure awareness of child processes.  Data is passed in a minimal way to allow all processes to update their knowledge while minimizing overhead.  This method has been validated on a small variety of systems and seems to perform better for systems of the size of OWP.  Obtaining optimal parallel scaling seems a difficult challenge while maintaining the sorted insertion ordering.  By relaxing the ordering one could enable batching which would enable more efficient parallelism.  This is a future work if needed.  It would seem this method could reasonably scale to 100 qubit chemical systems in runs that would take on the order of weeks.  For OWP, I anticipate on the order of hours (50k terms takes about 30 seconds with 17 cores, so roughly 8 hours for 500k terms in worst case).  

The preceding validations and benchmarks take around 10 minutes to run provided the Hamiltonians are cached.  (Caching simply skips the loading, JWT, and random subset selection for the OWP 50k validation.)

In [2]:
def compare_commuting_sets(sets1, sets2):
    if len(sets1) != len(sets2):
        print(f"Different number of sets: {len(sets1)} vs {len(sets2)}")
        return False
    
    def sort_key(commset):
        return (len(commset), str(sorted([str(term) for term in commset])))
    
    sets1_sorted = sorted(sets1, key=sort_key)
    sets2_sorted = sorted(sets2, key=sort_key)
    
    for i, (set1, set2) in enumerate(zip(sets1_sorted, sets2_sorted)):
        if len(set1) != len(set2):
            print(f"Set {i}: Different lengths {len(set1)} vs {len(set2)}")
            return False
        
        terms1 = sorted([str(term) for term in set1])
        terms2 = sorted([str(term) for term in set2])
        
        if terms1 != terms2:
            print(f"Set {i}: Different terms")
            return False
    
    return True

hdf5_path = 'Hamiltonians/monomer_eqb.hdf5'
hamiltonian, nqubits, nterms = load_h2o_hamiltonian(hdf5_path)

hamiltonian_cirq = qubit_operator_to_pauli_sum(hamiltonian)
cirq_qubits = sorted(hamiltonian_cirq.qubits)
blocks_int = [list(range(nqubits))]

terms = get_terms_ordered_by_abscoeff(hamiltonian)
terms = [t for t in terms if () not in t.terms.keys()]

print("Testing k=1")

blocks_cirq_k1 = [[q] for q in cirq_qubits]
blocks_int_k1 = [[i] for i in range(nqubits)]

sets1_k1 = get_si_sets(hamiltonian, blocks_cirq_k1, verbosity=0)
sets2_k1 = get_si_sets_v2(hamiltonian, blocks_int_k1, verbosity=0)
sets3_k1 = get_si_sets_v2_parallel(terms, blocks_int_k1, verbosity=0, num_workers=2)

print(f"Results:")
print(f"  get_si_sets:             {len(sets1_k1)} commuting groups")
print(f"  get_si_sets_v2:          {len(sets2_k1)} commuting groups")
print(f"  get_si_sets_v2_parallel: {len(sets3_k1)} commuting groups")

if compare_commuting_sets(sets1_k1, sets2_k1) and compare_commuting_sets(sets1_k1, sets3_k1):
    print("\n✓ PASS: All three implementations produce identical results for k=1")
else:
    print("\n✗ FAIL: Implementations produce different results for k=1")

print("Testing k=4")

blocks_cirq_k4 = [cirq_qubits[i:i+4] for i in range(0, nqubits, 4)]
blocks_int_k4 = [list(range(i, i+4)) for i in range(0, nqubits, 4)]
if nqubits % 4 != 0:
    blocks_int_k4[-1] = list(range(blocks_int_k4[-1][0], nqubits))
    blocks_cirq_k4[-1] = cirq_qubits[blocks_cirq_k4[-1][0].x:]

sets1_k4 = get_si_sets(hamiltonian, blocks_cirq_k4, verbosity=0)
sets2_k4 = get_si_sets_v2(hamiltonian, blocks_int_k4, verbosity=0)
sets3_k4 = get_si_sets_v2_parallel(terms, blocks_int_k4, verbosity=0, num_workers=2)

print(f"\nResults:")
print(f"  get_si_sets:             {len(sets1_k4)} commuting groups")
print(f"  get_si_sets_v2:          {len(sets2_k4)} commuting groups")
print(f"  get_si_sets_v2_parallel: {len(sets3_k4)} commuting groups")

if compare_commuting_sets(sets1_k4, sets2_k4) and compare_commuting_sets(sets1_k4, sets3_k4):
    print("\n✓ PASS: All three implementations produce identical results for k=4")
else:
    print("\n✗ FAIL: Implementations produce different results for k=4")

print("Testing k=N")

blocks_cirq_kN = [cirq_qubits]
blocks_int_kN = [list(range(nqubits))]

print(f"Blocks: {blocks_int_kN}")

sets1_kN = get_si_sets(hamiltonian, blocks_cirq_kN, verbosity=0)
sets2_kN = get_si_sets_v2(hamiltonian, blocks_int_kN, verbosity=0)
sets3_kN = get_si_sets_v2_parallel(terms, blocks_int_kN, verbosity=0, num_workers=2)

print(f"\nResults:")
print(f"  get_si_sets:             {len(sets1_kN)} commuting groups")
print(f"  get_si_sets_v2:          {len(sets2_kN)} commuting groups")
print(f"  get_si_sets_v2_parallel: {len(sets3_kN)} commuting groups")

if compare_commuting_sets(sets1_kN, sets2_kN) and compare_commuting_sets(sets1_kN, sets3_kN):
    print("\n✓ PASS: All three implementations produce identical results for k=N")
else:
    print("\n✗ FAIL: Implementations produce different results for k=N")

Loaded H2O Hamiltonian: 14 qubits, 1620 terms
Testing k=1
Results:
  get_si_sets:             443 commuting groups
  get_si_sets_v2:          443 commuting groups
  get_si_sets_v2_parallel: 443 commuting groups

✓ PASS: All three implementations produce identical results for k=1
Testing k=4

Results:
  get_si_sets:             250 commuting groups
  get_si_sets_v2:          250 commuting groups
  get_si_sets_v2_parallel: 250 commuting groups

✓ PASS: All three implementations produce identical results for k=4
Testing k=N
Blocks: [[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]]

Results:
  get_si_sets:             65 commuting groups
  get_si_sets_v2:          65 commuting groups
  get_si_sets_v2_parallel: 65 commuting groups

✓ PASS: All three implementations produce identical results for k=N


In [3]:
print("Testing H2O Hamiltonian with all three implementations")

hdf5_path = 'Hamiltonians/monomer_eqb.hdf5'
hamiltonian, nqubits, nterms = load_h2o_hamiltonian(hdf5_path)

hamiltonian_cirq = qubit_operator_to_pauli_sum(hamiltonian)
cirq_qubits = sorted(hamiltonian_cirq.qubits)
blocks = [cirq_qubits]
blocks_int = [list(range(nqubits))]

print(f"\nRunning get_si_sets...")
groups_v1 = get_si_sets(hamiltonian, blocks, verbosity=0)

print(f"Running get_si_sets_v2...")
groups_v2 = get_si_sets_v2(hamiltonian, blocks_int, verbosity=0)

print(f"Running get_si_sets_v2_parallel...")
terms = get_terms_ordered_by_abscoeff(hamiltonian)
terms = [t for t in terms if () not in t.terms.keys()]
groups_v2_parallel = get_si_sets_v2_parallel(terms, blocks_int, verbosity=0, num_workers=2)

print(f"\nResults:")
print(f"  get_si_sets:             {len(groups_v1)} groups")
print(f"  get_si_sets_v2:          {len(groups_v2)} groups")
print(f"  get_si_sets_v2_parallel: {len(groups_v2_parallel)} groups")

if len(groups_v1) == len(groups_v2) == len(groups_v2_parallel):
    print(f"\n✓ PASS: All three implementations produce {len(groups_v1)} groups")
else:
    print(f"\n✗ FAIL: Implementations produce different numbers of groups")

print("H2O Hamiltonian Timing Comparison")

print(f"\nRunning get_si_sets...")
start = time.time()
groups_v1 = get_si_sets(hamiltonian, blocks, verbosity=0)
t_v1 = time.time() - start
print(f"  Time: {t_v1:.3f}s, {len(groups_v1)} groups")

print(f"\nRunning get_si_sets_v2...")
start = time.time()
groups_v2 = get_si_sets_v2(hamiltonian, blocks_int, verbosity=0)
t_v2 = time.time() - start
speedup_v2 = t_v1 / t_v2
print(f"  Time: {t_v2:.3f}s, {len(groups_v2)} groups, speedup: {speedup_v2:.2f}x")

print(f"\nRunning get_si_sets_v2_parallel...")
start = time.time()
groups_v2_parallel = get_si_sets_v2_parallel(terms, blocks_int, verbosity=0, num_workers=2)
t_v2_parallel = time.time() - start
speedup_v2_parallel = t_v1 / t_v2_parallel
print(f"  Time: {t_v2_parallel:.3f}s, {len(groups_v2_parallel)} groups, speedup: {speedup_v2_parallel:.2f}x")

print(f"Summary:")
print(f"  get_si_sets:             {t_v1:.3f}s, {len(groups_v1)} groups (baseline)")
print(f"  get_si_sets_v2:          {t_v2:.3f}s, {len(groups_v2)} groups ({speedup_v2:.2f}x faster)")
print(f"  get_si_sets_v2_parallel: {t_v2_parallel:.3f}s, {len(groups_v2_parallel)} groups ({speedup_v2_parallel:.2f}x faster)")

if len(groups_v1) == len(groups_v2) == len(groups_v2_parallel):
    print(f"\n✓ PASS: All three implementations produce {len(groups_v1)} groups")
else:
    print(f"\n✗ FAIL: Implementations produce different numbers of groups")

Testing H2O Hamiltonian with all three implementations
Loaded H2O Hamiltonian: 14 qubits, 1620 terms

Running get_si_sets...
Running get_si_sets_v2...
Running get_si_sets_v2_parallel...

Results:
  get_si_sets:             65 groups
  get_si_sets_v2:          65 groups
  get_si_sets_v2_parallel: 65 groups

✓ PASS: All three implementations produce 65 groups
H2O Hamiltonian Timing Comparison

Running get_si_sets...
  Time: 58.241s, 65 groups

Running get_si_sets_v2...
  Time: 0.076s, 65 groups, speedup: 770.98x

Running get_si_sets_v2_parallel...
  Time: 0.317s, 65 groups, speedup: 183.51x
Summary:
  get_si_sets:             58.241s, 65 groups (baseline)
  get_si_sets_v2:          0.076s, 65 groups (770.98x faster)
  get_si_sets_v2_parallel: 0.317s, 65 groups (183.51x faster)

✓ PASS: All three implementations produce 65 groups


Here we see the parallel version slightly underperform.  At 0.76 seconds for the get_si_sets_v2, parallel overhead simply dominates any possible advantage.  As problems become harder, parallel advantage begins to emerge:

In [4]:
print('Testing OWP 50k terms subset - Correctness')

with open('tests/cached_50k_subset.pkl', 'rb') as f:
    P, nq = pickle.load(f)

terms = get_terms_ordered_by_abscoeff(P)
terms = [t for t in terms if () not in t.terms.keys()]
blocks = [list(range(nq))]

print(f'Testing {len(terms)} terms\n')

print('Running get_si_sets_v2...')
g_serial = get_si_sets_v2(P, blocks, verbosity=0)

print('Running get_si_sets_v2_parallel...')
g_parallel = get_si_sets_v2_parallel(terms, blocks, verbosity=0, num_workers=3, handshake_interval=75)

print(f'\nResults:')
print(f'  get_si_sets_v2:          {len(g_serial)} groups')
print(f'  get_si_sets_v2_parallel: {len(g_parallel)} groups')

if len(g_serial) == len(g_parallel):
    print(f'\n✓ PASS: Both implementations produce {len(g_serial)} groups')
else:
    print(f'\n✗ FAIL: Implementations produce different numbers of groups')

print('OWP 50k terms subset - Timing')

print(f'Testing {len(terms)} terms\n')

print('Running get_si_sets_v2...')
start = time.time()
g_serial = get_si_sets_v2(P, blocks, verbosity=0)
t_serial = time.time() - start
print(f'  Time: {t_serial:.3f}s, {len(g_serial)} groups')

print('\nRunning get_si_sets_v2_parallel...')
start = time.time()
g_parallel = get_si_sets_v2_parallel(terms, blocks, verbosity=0, num_workers=3, handshake_interval=75)
t_parallel = time.time() - start
speedup = t_serial / t_parallel
print(f'  Time: {t_parallel:.3f}s, {len(g_parallel)} groups, speedup: {speedup:.2f}x')

print(f'Summary:')
print(f'  get_si_sets_v2:          {t_serial:.3f}s, {len(g_serial)} groups (baseline)')
print(f'  get_si_sets_v2_parallel: {t_parallel:.3f}s, {len(g_parallel)} groups ({speedup:.2f}x faster)')


if len(g_serial) == len(g_parallel):
    print(f'\n✓ PASS: Both implementations produce {len(g_serial)} groups')
else:
    print(f'\n✗ FAIL: Implementations produce different numbers of groups')

Testing OWP 50k terms subset - Correctness
Testing 50000 terms

Running get_si_sets_v2...
Running get_si_sets_v2_parallel...

Results:
  get_si_sets_v2:          1189 groups
  get_si_sets_v2_parallel: 1189 groups

✓ PASS: Both implementations produce 1189 groups
OWP 50k terms subset - Timing
Testing 50000 terms

Running get_si_sets_v2...
  Time: 75.854s, 1189 groups

Running get_si_sets_v2_parallel...
  Time: 53.993s, 1189 groups, speedup: 1.40x
Summary:
  get_si_sets_v2:          75.854s, 1189 groups (baseline)
  get_si_sets_v2_parallel: 53.993s, 1189 groups (1.40x faster)

✓ PASS: Both implementations produce 1189 groups


vvv Don't forget to save the grouping! vvv

In [5]:
print('Full OWP Hamiltonian')

npz_path = 'Hamiltonians/owp_reactant.npz'
hamiltonian_owp, nqubits_owp, nterms_owp = load_owp_hamiltonian(npz_path)

terms_owp = get_terms_ordered_by_abscoeff(hamiltonian_owp)
terms_owp = [t for t in terms_owp if () not in t.terms.keys()]
blocks_owp = [list(range(nqubits_owp))]

print(f'\nProcessing {len(terms_owp)} terms with {nqubits_owp} qubits\n')

print('Running get_si_sets_v2_parallel with 16 workers, handshake_interval=75, chunk_divisor=2...')
start = time.time()
# groups_owp_serial = get_si_sets_v2(hamiltonian_owp, blocks_owp, verbosity=0)
groups_owp = get_si_sets_v2_parallel(
    terms_owp, 
    blocks_owp, 
    verbosity=0, 
    num_workers=16, 
    handshake_interval=75,
    chunk_divisor=2
)
t_owp_parallel = time.time() - start

print(f'\nCompleted in {t_owp_parallel:.3f}s')
print(f'Result: {len(groups_owp)} commuting groups')

Full OWP Hamiltonian
Loaded OWP Hamiltonian:
  Core energy: -479.76304624866236
  Number of orbitals: 22
  Number of electrons: 32
  H1 shape: (22, 22)
  H2 shape: (22, 22, 22, 22)
  FermionOperator terms: 937977
Applying Jordan-Wigner transformation...


KeyboardInterrupt: 